## Variable sharing

>.NET Interactive enables you to write code in multiple languages within a single notebook and in order to take advantage of those languages' different strengths, you might find it useful to share data between them. …You can share variables between .NET subkernels using the `#!share` magic command.
>
> —Jon Sequeira ([@jonsequitur](https://github.com/jonsequitur)), “[Variable sharing](https://github.com/dotnet/interactive/blob/main/docs/variable-sharing.md#variable-sharing)”


In [ ]:
#!share --help

#!share
  Share a .NET variable between subkernels

Usage:
  [options] #!share <name>

Arguments:
  <$|?|ConfirmPreference|DebugPreference|EnabledExperimentalFeatures|Error|ErrorActionPreference|ErrorView|ExecutionContext|false|FormatEnumerationLimit|HOME|Host|InformationPreference|InteractiveHost|IsCoreCLR|IsLinux|IsMacOS|IsWindows|MaximumHistoryCount|NestedPromptLevel|null|OutputEncoding|PID|PROFILE|ProgressPreference|PSCulture|PSDefaultParameterValues|PSEdition|PSEmailServer|PSHOME|PSSessionApplicationName|PSSessionConfigurationName|PSSessionOption|PSUICulture|PSVersionTable|PWD|ShellId|StackTrace|true|VerbosePreference|WarningPreference|WhatIfPreference|^>  The name of the variable to create in the destination kernel

Options:
  --from <csharp|fsharp|pwsh|value>  The name of the kernel where the variable has been previously declared
  -?, -h, --help                     Show help and usage information



In [ ]:
var fromCSharp = "This is from C#";

fromCSharp

This is from C#

In [ ]:
#!share --from csharp fromCSharp

let message = $"message: {fromCSharp}"

message

message: This is from C#

### the `#!value` command

>The `#!value` magic command is available to make it as easy as possible to get that text into a variable in your notebook. An important thing to know is that `#!value` is an alias to a subkernel designed just to hold values. This means that once you store something in it, you can access it from another subkernel using `#!share`.


In [ ]:
#!value --help

#!value
  Stores a value that can then be shared with other subkernels.

Usage:
  [options] #!value

Options:
  --name <name> (REQUIRED)                                      The name of the value to create. You can use #!share to retrieve this value from another subkernel.
  --from-file <from-file>                                       Specifies a file whose contents will be stored.
  --from-url <from-url>                                         Specifies a URL whose content will be stored.
  --mime-type <application/json|text/csv|text/html|text/plain>  A mime type for the value. If specified, displays the value immediately as an output using the specified mime type.
  -?, -h, --help                                                Show help and usage information



>The simplest way to use `#!value` is to paste some text into the cell. The text will be stored as a string, but unlike using a string literal in C#, F#, or PowerShell, there's no need to escape anything.


In [2]:
#!csharp
#!value --name someJson

{
    "what": "some JSON",
    "why": "to share it with another subkernel"
}

In [3]:
#!csharp
#!share someJson --from value

using System.Text.Json;

var jsonDoc = JsonDocument.Parse(someJson);

jsonDoc.Display("application/json")

`JsonDocument` does not have a `Display()` method. The `Display()` extension method is part of the [formatting](https://github.com/dotnet/interactive/blob/82a2545a4965305f8909e9950ab6e1e1ad4516af/docs/formatting.md) features of .NET Interactive:

>Formatting is invoked when values are displayed either implicitly (using a trailing expression or return statement), using the Display extension method, or using helper methods such as display.
>
> —Don Syme ([@dsymetweets](https://twitter.com/dsymetweets)) and Jon Sequeira ([@jonsequitur](https://github.com/jonsequitur)), “[Formatting](https://github.com/dotnet/interactive/blob/main/docs/formatting.md#formatting)”

There are two other ways to set a `#!value`:

1. `--from-file`
2. `--from-url`


In [ ]:
#!value --from-file ./dotnet-interactive-02-file.json --name fileJson

In [ ]:
#!share fileJson --from value

jsonDoc = JsonDocument.Parse(fileJson);

jsonDoc.Display("application/json")

In [4]:
#!csharp
#!value --from-url http://jsonplaceholder.typicode.com/users --name wildJson --mime-type application/json

>Regardless of which of these approaches you use, you can additionally choose to display the value in the notebook at the time of submission by using the `--mime-type` option. This accomplishes a few things. If your notebook frontend knows how to display that mime type, you can see it appropriately formatted…


In [ ]:
#!share wildJson --from value

jsonDoc = JsonDocument.Parse(wildJson);

jsonDoc.RootElement[0]
    .GetProperty("address")
    .GetProperty("geo")
    .Display("application/json")

In [ ]:
var csharpNow = new { now = DateTime.Now };

>Variable sharing has some limitations to be aware of. When sharing a variable with a subkernel where its compilation requirements aren't met, for example due to a missing using (C#) or open (F#) declaration, a custom type defined in the notebook, or a missing assembly reference, #!share will fail…


In [ ]:
#!share --from csharp csharpNow

let now = csharpNow


Error: System.InvalidOperationException: Unable to import type, <>f__AnonymousType0#9`1[System.DateTime].
 ---> System.InvalidOperationException: Unable to import type, <>f__AnonymousType0#9`1[System.DateTime].
   at FSharp.Compiler.Interactive.Shell.FsiDynamicCompiler.importReflectionType(FsiDynamicCompilerState istate, Type reflectionTy) in D:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 1363
   at FSharp.Compiler.Interactive.Shell.FsiDynamicCompiler.AddBoundValue(CompilationThreadToken ctok, ErrorLogger errorLogger, FsiDynamicCompilerState istate, String name, Object value) in D:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 1651
   --- End of inner exception stack trace ---
   at FSharp.Compiler.Interactive.Shell.FsiEvaluationSession.commitResult[a,b](FSharpChoice`2 res) in D:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 2925
   at FSharp.Compiler.Interactive.Shell.FsiEvaluationSession.AddBoundValue(String name, Object value) in D:\workspace\_work\1\s\src\fsharp\fsi\fsi.

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.240403+56a0e392fba305af06e72ce7dec885b2b6722c00Build date: 2021-08-08T21:10:43.6754968Zhttps://github.com/dotnet/interactive
